In [2]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

2022-03-24 11:34:28.271223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 11:34:28.271549: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
from RainPredictionMachine.data import CleanDataRpm
cleaner = CleanDataRpm()
df = cleaner.clean_data(1)

In [4]:
df.head(2).to_dict()

{'Chuva': {0: 0.0, 1: 0.0},
 'Pres': {0: 923.5, 1: 924.0},
 'Pres_max': {0: 923.5, 1: 924.0},
 'Pres_min': {0: 922.4, 1: 923.5},
 'Radiacao': {0: 0.0, 1: 0.0},
 'Temp': {0: 21.8, 1: 22.0},
 'Temp_orvalho': {0: 18.9, 1: 19.4},
 'Temp_max': {0: 22.8, 1: 22.3},
 'Temp_min': {0: 21.7, 1: 21.6},
 'Temp_orvalho_max': {0: 19.3, 1: 19.7},
 'Temp_orvalho_min': {0: 18.7, 1: 18.9},
 'Umid_max': {0: 84.0, 1: 86.0},
 'Umid_min': {0: 79.0, 1: 84.0},
 'Umid': {0: 84.0, 1: 85.0},
 'Dir_vento': {0: 164.0, 1: 348.0},
 'Rajada_vento': {0: 2.6, 1: 2.0},
 'Vel_vento': {0: 0.7, 1: 0.1},
 'Latitude': {0: -23.52388888, 1: -23.52388888},
 'Longitude': {0: -46.86944443, 1: -46.86944443},
 'Altitude': {0: 776.5, 1: 776.5},
 'datahora': {0: Timestamp('2020-01-01 00:00:00+0000', tz='UTC'),
  1: Timestamp('2020-01-01 01:00:00+0000', tz='UTC')},
 'classe_chuva': {0: 0, 1: 0}}

In [5]:
df.isnull().sum()

Chuva               0
Pres                0
Pres_max            0
Pres_min            0
Radiacao            0
Temp                0
Temp_orvalho        0
Temp_max            0
Temp_min            0
Temp_orvalho_max    0
Temp_orvalho_min    0
Umid_max            0
Umid_min            0
Umid                0
Dir_vento           0
Rajada_vento        0
Vel_vento           0
Latitude            0
Longitude           0
Altitude            0
datahora            0
classe_chuva        0
dtype: int64

In [6]:
def subsample_sequence(df, length):
    index = np.random.randint(0, df.shape[0] - length)
    df_sample = df.iloc[index:index+length]
    
    return df_sample

In [7]:
df_sample = subsample_sequence(df,48)
df_sample.head()

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid_min,Umid,Dir_vento,Rajada_vento,Vel_vento,Latitude,Longitude,Altitude,datahora,classe_chuva
6762,0.0,925.2,925.6,925.2,1448.90,28.0,17.7,28.2,26.4,19.1,...,51.0,54.0,66.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-10-08 18:00:00+00:00,0
6763,0.0,926.4,926.4,925.2,26.45,20.7,17.9,28.0,20.6,18.9,...,54.0,84.0,104.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-10-08 19:00:00+00:00,0
6764,0.0,927.1,927.1,926.4,144.75,19.7,17.5,20.7,19.6,17.9,...,84.0,87.0,85.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-10-08 20:00:00+00:00,0
6765,0.0,927.3,927.5,926.8,0.00,19.9,17.1,19.9,19.6,17.5,...,84.0,84.0,83.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-10-08 21:00:00+00:00,0
6766,0.0,926.7,927.3,926.6,0.00,20.2,17.0,20.2,19.9,17.3,...,82.0,82.0,28.0,0.0,0.0,-23.523889,-46.869444,776.5,2020-10-08 22:00:00+00:00,0


In [8]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample['classe_chuva'].iloc[length - 24:]
    
    X_sample = df_subsample.drop(columns= ['classe_chuva', 'datahora'])[0:length -24]
    X_sample = X_sample.values
   
    return np.array(X_sample), np.array(y_sample)

In [9]:
X,y = split_subsample_sequence(df,48)
X.shape

(24, 20)

In [10]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [11]:
X,y 

(array([[ 0.00000000e+00,  9.30300000e+02,  9.30300000e+02,
          9.30200000e+02,  0.00000000e+00,  2.79000000e+01,
          1.17000000e+01,  2.79000000e+01,  2.69000000e+01,
          1.20000000e+01,  1.01000000e+01,  3.90000000e+01,
          3.40000000e+01,  3.70000000e+01,  1.34000000e+02,
          0.00000000e+00,  0.00000000e+00, -2.35238889e+01,
         -4.68694444e+01,  7.76500000e+02],
        [ 0.00000000e+00,  9.30700000e+02,  9.30700000e+02,
          9.30300000e+02,  0.00000000e+00,  2.50000000e+01,
          1.30000000e+01,  2.83000000e+01,  2.50000000e+01,
          1.50000000e+01,  1.19000000e+01,  5.20000000e+01,
          3.70000000e+01,  4.70000000e+01,  1.88000000e+02,
          0.00000000e+00,  0.00000000e+00, -2.35238889e+01,
         -4.68694444e+01,  7.76500000e+02],
        [ 0.00000000e+00,  9.31200000e+02,  9.31200000e+02,
          9.30700000e+02,  0.00000000e+00,  2.28000000e+01,
          1.53000000e+01,  2.50000000e+01,  2.28000000e+01,
          1.

In [12]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [13]:
df.shape[0]*0.8

7027.200000000001

In [14]:
train_size = int(df.shape[0]*0.8)
df_train = df.iloc[:train_size,:]
df_test = df.iloc[train_size:,:]

In [15]:
df_train.value_counts(['classe_chuva'])

classe_chuva
0               6538
1                449
2                 33
3                  7
dtype: int64

In [16]:
df_test.shape

(1757, 22)

In [17]:
X_train,y_train = get_X_y(df_train,6000,72)

In [18]:
X_test,y_test = get_X_y(df_test,6000,72)

In [19]:
y_train.shape

(6000, 24)

In [20]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# label = LabelEncoder()
# y_enc = label.fit_transform(y.reshape((-1,)))
y_cat = to_categorical(y_train)
y_cat.shape

(6000, 24, 4)

In [21]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

norm = Normalization()
norm.adapt(X_train)

2022-03-24 11:35:40.776944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-24 11:35:40.777726: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-24 11:35:40.777814: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-9BMAD68): /proc/driver/nvidia/version does not exist
2022-03-24 11:35:40.880903: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(24, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, None, 20)         41        
 n)                                                              
                                                                 
 lstm (LSTM)                 (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 24)                264       
                                                                 
Total params: 3,795
Trainable params: 3,754
Non-trainable params: 41
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1,
         validation_split=0.2,
          callbacks=[es])

Epoch 1/10
150/150 [==============================] - 11s 46ms/step - loss: 0.4421 - accuracy: 0.0188 - val_loss: 0.2955 - val_accuracy: 0.0225
Epoch 2/10
150/150 [==============================] - 6s 39ms/step - loss: 0.2573 - accuracy: 0.0237 - val_loss: 0.2390 - val_accuracy: 0.0242
Epoch 3/10
150/150 [==============================] - 7s 49ms/step - loss: 0.2279 - accuracy: 0.0242 - val_loss: 0.2273 - val_accuracy: 0.0342
Epoch 4/10
150/150 [==============================] - 7s 49ms/step - loss: 0.2214 - accuracy: 0.0913 - val_loss: 0.2251 - val_accuracy: 0.0142
Epoch 5/10
150/150 [==============================] - 6s 37ms/step - loss: 0.2173 - accuracy: 0.0210 - val_loss: 0.2226 - val_accuracy: 0.0225
Epoch 6/10
150/150 [==============================] - 7s 50ms/step - loss: 0.2143 - accuracy: 0.0265 - val_loss: 0.2196 - val_accuracy: 0.0308
Epoch 7/10
150/150 [==============================] - 6s 43ms/step - loss: 0.2119 - accuracy: 0.0498 - val_loss: 0.2165 - val_accuracy: 0.033

In [ ]:
model.predict(X_test)

In [ ]:
# import pickle
# with open('df0.pickle', 'rb') as f:
#     df1 = pickle.load(f)